In [199]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
from datetime import timedelta

pd.options.display.max_columns = None

In [200]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
url2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
df_usconf = pd.read_csv(url,error_bad_lines = False)
df_usdead = pd.read_csv(url2,error_bad_lines = False)

In [201]:
to_fix = ['totalTestResultsIncrease','positiveIncrease','negativeIncrease']
actual = ['totalTestResults','positive','negative']
columns = df_testing.columns
def fill_func(states):
    for state in states:
        for col in range(0,len(to_fix)):
            cumsum = df_testing.loc[df_testing['Province_State']==state].sort_values(by='Date')[to_fix[col]].cumsum()
            actualsum = df_testing.loc[df_testing['Province_State']==state,['Date',actual[col]]].sort_values(by='Date')[actual[col]]
            comparison = cumsum.eq(actualsum)
            if comparison[comparison == False].count() > 0:
                Earliest_index = comparison[comparison == False].index[0]
                df_testing.iloc[Earliest_index,columns.get_loc(to_fix[col])] = df_testing.iloc[Earliest_index,columns.get_loc(actual[col])]            

In [202]:
url = 'https://covidtracking.com/api/v1/states/daily.csv'
df_testing = pd.read_csv(url,error_bad_lines = False)
df_testing = df_testing.rename(columns = {'date':'Date','state':'Province_State'})
df_testing[['totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative']] = df_testing[['totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative']].fillna(0)

fill_func(df_testing['Province_State'].unique())

df_testing['Date'] = pd.to_datetime(df_testing['Date'].astype(dtype = 'str'))

ca_testing = df_testing[['Date','Province_State','positive','negative','hospitalizedCurrently','inIcuCurrently', 'death', 'positiveIncrease','negativeIncrease','totalTestResults','totalTestResultsIncrease','deathIncrease']].loc[df_testing['Province_State'] =='CA'].copy().reset_index(drop=True)
ca_testing['Province_State'] = 'California'
ca_testing = ca_testing.sort_values(by = ['Province_State','Date']).reset_index(drop=True)

In [203]:
us_conf_daily = df_usconf.copy()
us_dead_daily = df_usdead.copy()

us_conf_daily.iloc[:,11:] = us_conf_daily.iloc[:,11:].diff(axis=1).fillna(0).astype('int')
us_dead_daily.iloc[:,12:] = us_dead_daily.iloc[:,12:].diff(axis=1).fillna(0).astype('int')

us_conf_daily = us_conf_daily.groupby(['Province_State'])[us_conf_daily.columns.to_list()[11:]].agg('sum').reset_index()
us_dead_daily = us_dead_daily.groupby(['Province_State'])[us_dead_daily.columns.to_list()[11:]].agg('sum').reset_index()

us_conf_daily = us_conf_daily.melt(id_vars = us_conf_daily.columns[:1], var_name = 'Date', value_name = 'Daily Confirmed Cases').sort_values(by = ['Province_State','Date']).reset_index(drop=True)
us_dead_daily = us_dead_daily.melt(id_vars = us_dead_daily.columns[:2], var_name = 'Date', value_name = 'Daily Dead').sort_values(by = ['Province_State','Date']).reset_index(drop=True)

us_daily = us_conf_daily.merge(us_dead_daily[['Province_State','Population','Date']], on = ['Province_State','Date'], how = 'left', suffixes = (False, False))
us_daily = us_daily[['Province_State','Population','Date']]
us_daily['Date'] = pd.to_datetime(us_daily['Date'])
us_daily = us_daily.sort_values(by = ['Province_State','Date']).reset_index(drop=True)

In [5]:
#there will be discrepancies between the daily confirmed cases from JHU and from CDHP.
#Should probably use all CDHP metrics for CA analysis and only population from JHU
ca_daily = us_daily.loc[us_daily['Province_State']=='California'].copy().reset_index(drop=True)
ca_daily = ca_daily.merge(ca_testing[['Province_State', 'Date','totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative','hospitalizedCurrently','inIcuCurrently']], on = ['Province_State', 'Date'], how = 'left', suffixes = (False, False))

In [6]:
ca_daily.to_csv(r'X:\AC\Documents\Datasets\CA_daily_pivot.csv')

# Data Exploration

In [14]:
ca_daily[70:100]

,Province_State,Population,Date,totalTestResultsIncrease,positiveIncrease,negativeIncrease,totalTestResults,positive,negative,hospitalizedCurrently,inIcuCurrently
70,California,39512223,2020-04-01,673.0,673.0,0.0,29927.0,8155.0,21772.0,1855.0,774.0
71,California,39512223,2020-04-02,3073.0,1036.0,2037.0,33000.0,9191.0,23809.0,1922.0,816.0
72,California,39512223,2020-04-03,2300.0,1510.0,790.0,35300.0,10701.0,24599.0,2188.0,901.0
73,California,39512223,2020-04-04,78400.0,1325.0,77075.0,113700.0,12026.0,101674.0,2300.0,1008.0
74,California,39512223,2020-04-05,2833.0,1412.0,1421.0,116533.0,13438.0,103095.0,2398.0,1040.0
75,California,39512223,2020-04-06,898.0,898.0,0.0,117431.0,14336.0,103095.0,2509.0,1085.0
76,California,39512223,2020-04-07,13798.0,1529.0,12269.0,131229.0,15865.0,115364.0,2611.0,1108.0
77,California,39512223,2020-04-08,13035.0,1092.0,11943.0,144264.0,16957.0,127307.0,2714.0,1154.0
78,California,39512223,2020-04-09,19236.0,1352.0,17884.0,163500.0,18309.0,145191.0,2825.0,1132.0
79,California,39512223,2020-04-10,1363.0,1163.0,200.0,164863.0,19472.0,145391.0,2897.0,1145.0


In [21]:
ca_daily['totalTestResultsIncrease'].isna().sum()

42

In [7]:
most_recent = df_testing.loc[(df_testing['Province_State']=='CA')]['Date'].max()
last14 = [most_recent - timedelta(days=x) for x in range(14)]
most_recent = df_testing.loc[(df_testing['Province_State']=='CA') & (df_testing['Date']==most_recent)]
last14 = df_testing.loc[(df_testing['Province_State']=='CA') & (df_testing['Date'].isin(last14))]

current_hosp = most_recent['hospitalizedCurrently'].iloc[0].astype(int)
perc_pos = (most_recent['positive']/most_recent['totalTestResults']).iloc[0] * 100
total_tests = most_recent['totalTestResults'].iloc[0]
population = ca_daily['Population'][0]
testsper1000 = total_tests/population*1000

last14tests = last14['totalTestResultsIncrease'].sum()
last14perc_pos = last14['positiveIncrease'].sum()/last14tests * 100

print('California Stats as of: ', pd.datetime.strftime(most_recent['Date'].max(), '%m/%d/%y'))
print('Total population in CA: %d' %population)
print('\n')
print('Total tests conducted: %d' %total_tests)
print('Total tests conducted per 1000 people in CA: %.2f' %testsper1000)
print('Percentage of total tests confirmed positive: %.2f' %perc_pos,'%')
print('Percentage of total tests confirmed negative: %.2f' %(100 - perc_pos),'%')
print(f'Currently hospitalized: {current_hosp}')
print('\n')
print('Last 14 days:')
print('Total tests conducted in last 14 days: %d' %last14tests)
print('Percentage of tests in last 14 days confirmed positive: %.2f' %last14perc_pos,'%')

California Stats as of:  07/21/20
Total population in CA: 39512223


Total tests conducted: 6536932
Total tests conducted per 1000 people in CA: 165.44
Percentage of total tests confirmed positive: 6.13 %
Percentage of total tests confirmed negative: 93.87 %
Currently hospitalized: 8681


Last 14 days:
Total tests conducted in last 14 days: 1640562
Percentage of tests in last 14 days confirmed positive: 7.50 %


# Appendix

In [ ]:
#apply date format and convert to string
.apply(lambda x: pd.datetime.strftime(x, '%m/%d/%y'))

In [75]:
columns = test_df.columns
test_df.iloc[7694,columns.get_loc('totalTestResultsIncrease')] = 1
a = np.cumsum(test_df.loc[test_df['Province_State']=='TX'].sort_values(by='Date')['totalTestResultsIncrease'].values,dtype=int)
b = test_df.loc[test_df['Province_State']=='TX'].sort_values(by='Date')['totalTestResults'].values
np.array_equal(a,b)

True

In [ ]:
#filling missing starting values. May want to do this as function for all states
df_testing.loc[(df_testing['Province_State']=='CA') & (df_testing['Date']==20200304), ['positiveIncrease','negativeIncrease','totalTestResultsIncrease']] = df_testing.loc[(df_testing['Province_State']=='CA') & (df_testing['Date']==20200304), ['positive','negative','totalTestResults']].values.tolist()

In [ ]:
#Update function with secondary check
to_fix = ['totalTestResultsIncrease','positiveIncrease','negativeIncrease']
actual = ['totalTestResults','positive','negative']
columns = df_testing.columns
def fill_func(states):
    for state in states:
        print(state)
        for col in range(0,len(to_fix)):
            cumsum = df_testing.loc[df_testing['Province_State']==state].sort_values(by='Date')[to_fix[col]].cumsum()
            actualsum = df_testing.loc[df_testing['Province_State']==state,['Date',actual[col]]].sort_values(by='Date')[actual[col]]
            comparison = cumsum.eq(actualsum)
            print(cumsum.iloc[0], actualsum.iloc[0], comparison.iloc[0])
            if comparison[comparison == False].count() > 0:
                Earliest_index = comparison[comparison == False].index[0]
                df_testing.iloc[Earliest_index,columns.get_loc(to_fix[col])] = df_testing.iloc[Earliest_index,columns.get_loc(actual[col])]
                print('Fixed in first: ', state,to_fix[col],actual[col])
            else:
                print('Arrays are equal')
            cumsum = df_testing.loc[df_testing['Province_State']==state].sort_values(by='Date')[to_fix[col]].cumsum()
            actualsum = df_testing.loc[df_testing['Province_State']==state,['Date',actual[col]]].sort_values(by='Date')[actual[col]]
            comparison = cumsum.eq(actualsum)
            if comparison[comparison == False].count() == 0:
                print('Confirmed arrays are fixed')
            else:
                print('Failed check: ', state,to_fix[col],actual[col] ,'\n')

In [15]:
#['Province_State', 'Date','totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative','hospitalizedCurrently','inIcuCurrently']
#hospitalized currently missing values from american samoa, Hawaii, Kansas, northern marinas, virginia
#inincu currently missing values from 28 states/provinces
df_testing.loc[df_testing['Date']=='2020-07-21',['Province_State', 'Date','totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative','hospitalizedCurrently','inIcuCurrently']]

,Province_State,Date,totalTestResultsIncrease,positiveIncrease,negativeIncrease,totalTestResults,positive,negative,hospitalizedCurrently,inIcuCurrently
0,AK,2020-07-21,3040.0,111.0,2929.0,178626,2525.0,176101.0,32.0,NaN
1,AL,2020-07-21,6593.0,1467.0,5126.0,600381,70358.0,530023.0,1547.0,NaN
2,AR,2020-07-21,6819.0,728.0,6091.0,439635,34655.0,404980.0,488.0,NaN
3,AS,2020-07-21,0.0,0.0,0.0,1037,0.0,1037.0,NaN,NaN
4,AZ,2020-07-21,14143.0,3500.0,10643.0,807666,148683.0,658983.0,3041.0,865.0
5,CA,2020-07-21,122611.0,9231.0,113380.0,6536932,400769.0,6136163.0,8681.0,2226.0
6,CO,2020-07-21,9130.0,424.0,8706.0,451299,40566.0,410733.0,386.0,NaN
7,CT,2020-07-21,5544.0,41.0,5503.0,662837,48096.0,614741.0,62.0,NaN
8,DC,2020-07-21,3971.0,88.0,3883.0,155711,11427.0,144284.0,83.0,19.0
9,DE,2020-07-21,5352.0,227.0,5125.0,158347,13746.0,144601.0,58.0,11.0


In [204]:
cumsum = df_testing.loc[df_testing['Province_State']=='NV'].sort_values(by='Date')['totalTestResultsIncrease'].cumsum()
actualsum = df_testing.loc[df_testing['Province_State']=='NV',['Date','totalTestResults']].sort_values(by='Date')['totalTestResults']
check = cumsum.eq(actualsum)
check[check==False].sum()

0